In [82]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 22: Slam Shuffle ---</h2><p>There isn't much to do while you wait for the droids to repair your ship.  At least you're drifting in the right direction.  You decide to practice a new <a href="https://en.wikipedia.org/wiki/Shuffling">card shuffle</a> you've been working on.</p>
<p>Digging through the ship's storage, you find a deck of <em>space cards</em>! Just like <span title="What do you mean, you've never heard of space cards? They're all the rage in Zozo.">any deck of space cards</span>, there are 10007 cards in the deck numbered <code>0</code> through <code>10006</code>. The deck must be new - they're still in <em>factory order</em>, with <code>0</code> on the top, then <code>1</code>, then <code>2</code>, and so on, all the way through to <code>10006</code> on the bottom.</p>
<p>You've been practicing three different <em>techniques</em> that you use while shuffling. Suppose you have a deck of only 10 cards (numbered <code>0</code> through <code>9</code>):</p>
<p><em>To <code>deal into new stack</code></em>, create a new stack of cards by dealing the top card of the deck onto the top of the new stack repeatedly until you run out of cards:</p>
<pre><code>Top          Bottom
0 1 2 3 4 5 6 7 8 9   Your deck
                      New stack

1 2 3 4 5 6 7 8 9 Your deck
0 New stack

    2 3 4 5 6 7 8 9   Your deck
                1 0   New stack

      3 4 5 6 7 8 9   Your deck
              2 1 0   New stack

Several steps later...

                  9   Your deck

8 7 6 5 4 3 2 1 0 New stack

                      Your deck

9 8 7 6 5 4 3 2 1 0 New stack
</code></pre>

<p>Finally, pick up the new stack you've just created and use it as the deck for the next technique.</p>
<p><em>To <code>cut N</code> cards</em>, take the top <code>N</code> cards off the top of the deck and move them as a single unit to the bottom of the deck, retaining their order. For example, to <code>cut 3</code>:</p>
<pre><code>Top          Bottom
0 1 2 3 4 5 6 7 8 9   Your deck

      3 4 5 6 7 8 9   Your deck

0 1 2 Cut cards

3 4 5 6 7 8 9 Your deck
0 1 2 Cut cards

3 4 5 6 7 8 9 0 1 2 Your deck
</code></pre>

<p>You've also been getting pretty good at a version of this technique where <code>N</code> is negative! In that case, cut (the absolute value of) <code>N</code> cards from the bottom of the deck onto the top.  For example, to <code>cut -4</code>:</p>
<pre><code>Top          Bottom
0 1 2 3 4 5 6 7 8 9   Your deck

0 1 2 3 4 5 Your deck
6 7 8 9 Cut cards

        0 1 2 3 4 5   Your deck

6 7 8 9 Cut cards

6 7 8 9 0 1 2 3 4 5 Your deck
</code></pre>

<p><em>To <code>deal with increment N</code></em>, start by clearing enough space on your table to lay out all of the cards individually in a long line.  Deal the top card into the leftmost position. Then, move <code>N</code> positions to the right and deal the next card there. If you would move into a position past the end of the space on your table, wrap around and keep counting from the leftmost card again.  Continue this process until you run out of cards.</p>
<p>For example, to <code>deal with increment 3</code>:</p>
<pre><code>
0 1 2 3 4 5 6 7 8 9   Your deck
. . . . . . . . . .   Space on table
^                     Current position

Deal the top card to the current position:

1 2 3 4 5 6 7 8 9 Your deck
0 . . . . . . . . . Space on table
^ Current position

Move the current position right 3:

1 2 3 4 5 6 7 8 9 Your deck
0 . . . . . . . . . Space on table
^ Current position

Deal the top card:

    2 3 4 5 6 7 8 9   Your deck

0 . . 1 . . . . . . Space on table
^ Current position

Move right 3 and deal:

      3 4 5 6 7 8 9   Your deck

0 . . 1 . . 2 . . . Space on table
^ Current position

Move right 3 and deal:

        4 5 6 7 8 9   Your deck

0 . . 1 . . 2 . . 3 Space on table
^ Current position

Move right 3, wrapping around, and deal:

          5 6 7 8 9   Your deck

0 . 4 1 . . 2 . . 3 Space on table
^ Current position

And so on:

0 7 4 1 8 5 2 9 6 3 Space on table
</code></pre>

<p>Positions on the table which already contain cards are still counted; they're not skipped.  Of course, this technique is carefully designed so it will never put two cards in the same position or leave a position empty.</p>
<p>Finally, collect the cards on the table so that the leftmost card ends up at the top of your deck, the card to its right ends up just below the top card, and so on, until the rightmost card ends up at the bottom of the deck.</p>
<p>The complete shuffle process (your puzzle input) consists of applying many of these techniques.  Here are some examples that combine techniques; they all start with a <em>factory order</em> deck of 10 cards:</p>
<pre><code>deal with increment 7
deal into new stack
deal into new stack
Result: 0 3 6 9 2 5 8 1 4 7
</code></pre>
<pre><code>cut 6
deal with increment 7
deal into new stack
Result: 3 0 7 4 1 8 5 2 9 6
</code></pre>
<pre><code>deal with increment 7
deal with increment 9
cut -2
Result: 6 3 0 7 4 1 8 5 2 9
</code></pre>
<pre><code>deal into new stack
cut -2
deal with increment 7
cut 8
cut -4
deal with increment 7
cut 3
deal with increment 9
deal with increment 3
cut -1
Result: 9 2 5 8 1 4 7 0 3 6
</code></pre>
<p>Positions within the deck count from <code>0</code> at the top, then <code>1</code> for the card immediately below the top card, and so on to the bottom.  (That is, cards start in the position matching their number.)</p>
<p>After shuffling your <em>factory order</em> deck of 10007 cards, <em>what is the position of card <code>2019</code>?</em></p>
</article>


In [83]:
tests = [
    {
        "name": "Example 1",
        "cards": 10,
        "techniques": """
            deal with increment 7
            deal into new stack
            deal into new stack
        """,
        "expected": "0 3 6 9 2 5 8 1 4 7",
    },
    {
        "name": "Example 2",
        "cards": 10,
        "techniques": """
            cut 6
            deal with increment 7
            deal into new stack
        """,
        "expected": "3 0 7 4 1 8 5 2 9 6",
    },
    {
        "name": "Example 3",
        "cards": 10,
        "techniques": """
            deal with increment 7
            deal with increment 9
            cut -2
        """,
        "expected": "6 3 0 7 4 1 8 5 2 9",
    },
    {
        "name": "Example 4",
        "cards": 10,
        "techniques": """
            deal into new stack
            cut -2
            deal with increment 7
            cut 8
            cut -4
            deal with increment 7
            cut 3
            deal with increment 9
            deal with increment 3
            cut -1
        """,
        "expected": "9 2 5 8 1 4 7 0 3 6",
    },
]


class CardDeck:
    def __init__(self, cards: int) -> None:
        self.deck = list(range(cards))

    def __repr__(self) -> str:
        return " ".join(str(c) for c in self.deck)

    def deal_into_new_stack(self) -> None:
        self.deck = self.deck[::-1]

    def deal_with_increment(self, inc: int) -> None:
        new_deck = [0] * len(self.deck)
        for i, card in enumerate(self.deck):
            new_deck[i * inc % len(self.deck)] = card
        self.deck = new_deck

    def cut(self, n: int) -> None:
        self.deck = self.deck[n:] + self.deck[:n]


# Test Cardeck
cd = CardDeck(10)
cd.deal_into_new_stack()
assert cd.deck == [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

cd = CardDeck(10)
cd.cut(3)
assert cd.deck == [3, 4, 5, 6, 7, 8, 9, 0, 1, 2]

cd = CardDeck(10)
cd.cut(-4)
assert cd.deck == [6, 7, 8, 9, 0, 1, 2, 3, 4, 5]

cd = CardDeck(10)
cd.deal_with_increment(3)
assert cd.deck == [0, 7, 4, 1, 8, 5, 2, 9, 6, 3]


def TestRunner(cards: int, techniques: str) -> str:
    deck = PartI(cards, techniques)
    return str(deck)


def PartI(cards, techniques) -> CardDeck:
    deck = CardDeck(cards)
    for tech in techniques.strip().splitlines():
        match tech.strip().split():
            case ["deal", "into", "new", "stack"]:
                deck.deal_into_new_stack()
            case ["deal", "with", "increment", inc]:
                deck.deal_with_increment(int(inc))
            case ["cut", n]:
                deck.cut(int(n))
    return deck


run_tests_params(TestRunner, tests)


Test Example 1 passed, for TestRunner.
Test Example 2 passed, for TestRunner.
Test Example 3 passed, for TestRunner.
Test Example 4 passed, for TestRunner.
Success


In [84]:
with open("../input/day22.txt") as f:
    puzzle = f.read()


print(f"Part I: {PartI(10_007, puzzle).deck.index(2019)}")

Part I: 6326


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>6326</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>After a while, you realize your shuffling skill won't improve much more with merely a single deck of cards.  You ask every 3D printer on the ship to make you some more cards while you check on the ship repairs.  While reviewing the work the droids have finished so far, you think you see <a href="https://en.wikipedia.org/wiki/Halley%27s_Comet">Halley's Comet</a> fly past!</p>
<p>When you get back, you discover that the 3D printers have combined their power to create for you a single, giant, brand new, <em>factory order</em> deck of <em><code>119315717514047</code> space cards</em>.</p>
<p>Finally, a deck of cards worthy of shuffling!</p>
<p>You decide to apply your complete shuffle process (your puzzle input) to the deck <em><code>101741582076661</code> times in a row</em>.</p>
<p>You'll need to be careful, though - one wrong move with this many cards and you might <em>overflow</em> your entire ship!</p>
<p>After shuffling your new, giant, <em>factory order</em> deck that many times, <em>what number is on the card that ends up in position <code>2020</code>?</em></p>
</article>

</main>


In [91]:
class CardDeckII:
    def __init__(self, cards: int, track: int) -> None:
        self.cards = cards
        self.track = track

    def __repr__(self) -> str:
        return " ".join(str(c) for c in self.cards)

    def deal_into_new_stack(self) -> None:
        self.track = self.cards - self.track - 1

    def deal_with_increment(self, inc: int) -> None:
        self.track = self.track * inc % self.cards

    def cut(self, n: int) -> None:
        if n >= 0:
            if self.track < n:
                self.track += self.cards - n
            else:
                self.track -= n
        else:
            if self.track < self.cards + n:
                self.track -= n
            else:
                self.track -= self.cards + n

        self.track %= self.cards


def PartII(cards, track, techniques, cycles=1) -> int:
    deck = CardDeckII(cards, track)
    techniques = techniques.strip().splitlines()
    for _ in range(cycles):
        for tech in techniques:
            match tech.strip().split():
                case ["deal", "into", "new", "stack"]:
                    deck.deal_into_new_stack()
                case ["deal", "with", "increment", inc]:
                    deck.deal_with_increment(int(inc))
                case ["cut", n]:
                    deck.cut(int(n))

    return deck.track


tests = [
    {
        "name": "Example 1",
        "cards": 10,
        "track": 7,
        "techniques": """
            deal with increment 7
            deal into new stack
            deal into new stack
        """,
        "expected": 9,
    },
    {
        "name": "Example 2",
        "cards": 10,
        "track": 7,
        "techniques": """
            cut 6
            deal with increment 7
            deal into new stack
        """,
        "expected": 2,
    },
    {
        "name": "Example 3",
        "cards": 10,
        "track": 7,
        "techniques": """
            deal with increment 7
            deal with increment 9
            cut -2
        """,
        "expected": 3,
    },
    {
        "name": "Example 4",
        "cards": 10,
        "track": 7,
        "techniques": """
            deal into new stack
            cut -2
            deal with increment 7
            cut 8
            cut -4
            deal with increment 7
            cut 3
            deal with increment 9
            deal with increment 3
            cut -1
        """,
        "expected": 6,
    },
    {
        "name": "Example 5 Deal new Stack",
        "cards": 10,
        "track": 4,
        "techniques": """
            deal into new stack
        """,
        "expected": 5,
    },
    {
        "name": "Example 6 cut 3 track 2",
        "cards": 10,
        "track": 2,
        "techniques": """
            cut 3
        """,
        "expected": 9,
    },
    {
        "name": "Example 7 cut 3 track 3",
        "cards": 10,
        "track": 3,
        "techniques": """
            cut 3
        """,
        "expected": 0,
    },
    {
        "name": "Example 8 cut 3 track 0",
        "cards": 10,
        "track": 0,
        "techniques": """
            cut 3
        """,
        "expected": 7,
    },
    {
        "name": "Example 9 cut -4 track 5",
        "cards": 10,
        "track": 5,
        "techniques": """
            cut -4
        """,
        "expected": 9,
    },
    {
        "name": "Example 9 cut -4 track 9",
        "cards": 10,
        "track": 9,
        "techniques": """
            cut -4
        """,
        "expected": 3,
    },
]


run_tests_params(PartII, tests)

# test
assert PartII(10_007, 2019, puzzle) == 6326

# print(
#     f"Part II: {PartII(119315717514047, 2020, puzzle, 101741582076661)}"
# )  Too slow offcourse


Test Example 1 passed, for PartII.
Test Example 2 passed, for PartII.
Test Example 3 passed, for PartII.
Test Example 4 passed, for PartII.
Test Example 5 Deal new Stack passed, for PartII.
Test Example 6 cut 3 track 2 passed, for PartII.
Test Example 7 cut 3 track 3 passed, for PartII.
Test Example 8 cut 3 track 0 passed, for PartII.
Test Example 9 cut -4 track 5 passed, for PartII.
Test Example 9 cut -4 track 9 passed, for PartII.
Success


In [92]:
# My math has too much faded away for this 😃
# Solution from https://github.com/metalim/adventofcode.2019.python/blob/master/22_cards_shuffle.ipynb
# convert rules to linear polynomial.
# (g∘f)(x) = g(f(x))
def parse(L, rules):
    rules = [l.strip() for l in rules.strip().splitlines()]
    a, b = 1, 0
    for s in rules[::-1]:
        if s == "deal into new stack":
            a = -a
            b = L - b - 1
            continue
        if s.startswith("cut"):
            n = int(s.split(" ")[1])
            b = (b + n) % L
            continue
        if s.startswith("deal with increment"):
            n = int(s.split(" ")[3])
            z = pow(n, L - 2, L)  # == modinv(n,L)
            a = a * z % L
            b = b * z % L
            continue
        raise Exception("unknown rule", s)
    return a, b


# modpow the polynomial: (ax+b)^m % n
# f(x) = ax+b
# g(x) = cx+d
# f^2(x) = a(ax+b)+b = aax + ab+b
# f(g(x)) = a(cx+d)+b = acx + ad+b
def polypow(a, b, m, n):
    if m == 0:
        return 1, 0
    if m % 2 == 0:
        return polypow(a * a % n, (a * b + b) % n, m // 2, n)
    else:
        c, d = polypow(a, b, m - 1, n)
        return a * c % n, (a * d + b) % n


def PartII_v2(L, N, pos, rules):
    a, b = parse(L, rules)
    a, b = polypow(a, b, N, L)
    return (pos * a + b) % L


print(f"Part II: {PartII_v2(119315717514047, 101741582076661, 2020, puzzle)}")

Part II: 40522432670594


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>40522432670594</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
